# --- Day 7: No Space Left On Device ---



In [1]:
# Load data
filename = "inputs/day7-example.txt"
with open(filename) as infile:
    terminal_output = [line.strip() for line in infile]
    

In [2]:
class Tree(object):
    def __init__(self, name='/', children=None, files=None):
        self.name = name
        self.children = []
        self.files = []
        self.size_files = 0
        self.size_subdirs = 0
        
        if children is not None:
            for child in children:
                self.add_child(child)
                
        if files is not None:
            for file in files:
                self.add_file(file)     
                
    def __repr__(self):
        return self.name
    
    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)
        
    def add_file(self, file):
        self.files.append(file)        
        self.size_files += file.get(list(file.keys())[0])


def traverse_filesystem(terminal_data):
    # Sketch of how to solve it...
    # 
    # Parse terminal data into a filesystem tree
    #
    # A node is defined by the sublist of terminal data 
    # starting with a pair of entries
    #     ["$ cd DIRECTORY", "$ ls"] 
    # ending with the corresponding "$ cd .." or end of 
    # the list (if there are no remaining commands)
    #
    # This means we know the number of nodes - it is the 
    # number of occurrences of "$ ls" in the terminal data
    
        
    file_tree = Tree()
    

    def process_node(file_tree, terminal_data, small_subdirs_total):
        
        if len(terminal_data) > 0:
            
            # @TODO: return parent if found "cd .."
            
            #print(terminal_data)
            
            # Stuff that happens when we're processing a node ...
            # Find first occurrence of "$ cd DIRECTORY" followed by "$ ls"
            indices_first_cd = [i for i in range(0, len(terminal_data)-1) if terminal_data[i] == f"$ cd {file_tree.name}" and terminal_data[i+1] == "$ ls"]
            index_first_cd = min(indices_first_cd) if len(indices_first_cd) > 0 else 0
            indices_next_command = [i for i in range(index_first_cd+2, len(terminal_data)-1) if terminal_data[i].startswith("$ ")]            
            index_next_command = min(indices_next_command) if len(indices_next_command) > 0 else len(terminal_data)

            node_listing = terminal_data[index_first_cd+2:index_next_command].copy()
            #print(node_listing)
            node_files = [x for x in node_listing if not x.startswith("dir ")]
            node_dirs = [x for x in node_listing if x.startswith("dir ")]
            
            #print(f"directory: {file_tree.name}")
            #print(f"files: {node_files}")
            #print(f"subdirs: {node_dirs}")
            #file_tree = Tree(name = terminal_data[index_first_cd].split("cd ")[1])
            for file in node_files:
                file_tree.add_file({file.split(" ")[1]: int(file.split(" ")[0])})
                
            # if len(node_dirs) == 0:
            #     print("this time")
            #     del terminal_data[index_first_cd:index_next_command]
            
            dir_size = file_tree.size_files
            
            for subdir in node_dirs:
                subdir_tree, terminal_data, subdir_size, small_subdirs_total = process_node(Tree(name = subdir.split("dir ")[1]), terminal_data[index_next_command-1:], small_subdirs_total)
                dir_size += subdir_size
                file_tree.add_child(subdir_tree)
                
            #print(f"dir_size: {dir_size}")
            
            if dir_size <= 100000:
                small_subdirs_total += dir_size
                

        
        return file_tree, terminal_data, dir_size, small_subdirs_total

    small_subdirs_total=0
    file_tree, _, _, small_subdirs_total = process_node(file_tree, terminal_data, small_subdirs_total)
    
    print(f"small_subdirs_total: {small_subdirs_total}")
            
                
    
    return file_tree
    
tree = traverse_filesystem(terminal_output.copy())
# print(tree.name)
# print(tree.files)
# print(tree.size_files)


small_subdirs_total: 95437


In [3]:
terminal_output

['$ cd /',
 '$ ls',
 'dir a',
 '14848514 b.txt',
 '8504156 c.dat',
 'dir d',
 '$ cd a',
 '$ ls',
 'dir e',
 '29116 f',
 '2557 g',
 '62596 h.lst',
 '$ cd e',
 '$ ls',
 '584 i',
 '$ cd ..',
 '$ cd ..',
 '$ cd d',
 '$ ls',
 '4060174 j',
 '8033020 d.log',
 '5626152 d.ext',
 '7214296 k']